In [1]:
import numpy as np
import os
from tqdm import tqdm
import requests
import matplotlib.pyplot as plt
import json

In [2]:
'''
Funcão utilizada para computar o valor de ajuste de cada solução da atual
população. Retorna a lista de [chomossomo,valor_de_ajuste] e prob_fits que
contém a probabilidade de cada cromossomo para a fase de seleção
'''

def fitness_function(population):
    evaluated = []
    for chromosome in population:
        resp = requests.get('https://aydanomachado.com/mlclass/02_Optimization.php?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}&dev_key=Andre%20Santos'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
        #resp = requests.get('http://localhost:8080/antenna/simulate?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
        resp = resp.text
        resp = resp.replace("'", "\"")
        resp = json.loads(resp)
        resp = resp['gain']
        #resp = float(resp.split('\n')[0])
        evaluated.append([chromosome,resp])
    evaluated.sort(key=lambda x:x[1], reverse=True)
    #prob_fits = [x[1] for x in evaluated]
    #prob_fits = np.exp(prob_fits) / np.sum(np.exp(prob_fits))
    return evaluated#,prob_fits

In [3]:
'''
Função que faz o cruzamento de dois cromossomos dados, retorna dois cromossomos filhos
'''
def crossover(father1,father2):
    
    child1 = [np.hstack((father1[0][:3],father2[0][3:])),0.0]
    child2 = [np.hstack((father2[0][:3],father1[0][3:])),0.0]
    
    #child1 = [np.hstack((father1[0][:2],father2[0][2:4],father1[0][4:])),0.0]
    #child2 = [np.hstack((father2[0][:2],father1[0][2:4],father2[0][4:])),0.0]

    return child1,child2

In [4]:
'''
Função que aplicaa mutação a um cromossomo dado, a probabilidade de mutação para um 
determinado cromossomo é de 5%
'''
def mutation(new_generation):

    prob_mutation = 20
    muted = []
    for chrom in new_generation:
        if np.random.randint(0,100) <= prob_mutation:
            position = np.random.randint(0,5)
            chrom[0][position] = np.random.randint(0,359)
            muted.append(chrom[0])
        else:
            muted.append(chrom[0])
    return muted

In [5]:
def selection(ordened_population,n_parents=6):
    selected = ordened_population[:n_parents]
    next_generation = []
    for i in range(0,n_parents-1):
        
        child1,child2 = crossover(selected[i],selected[i+1])
        next_generation.append(child1)
        next_generation.append(child2)
    
    return next_generation

In [6]:
def genetic_algorithm(population_init,n_gen):
    
    current_gen = population_init
    best_config = None
    best_value_fits = -float('inf')
    
    for gen in tqdm(range(n_gen)):
        current_gen = fitness_function(current_gen)
        
        if best_value_fits < current_gen[0][1]:
            best_config = current_gen[0][0]
            best_value_fits = current_gen[0][1]

        selected = selection(current_gen)
        muted = mutation(selected)
        
        next_gen = muted
        current_gen = next_gen
    
    return best_config,best_value_fits 

In [7]:
init_pop = np.random.randint(0,359,(10,6))
n_gen = 20
best_config, best_value = genetic_algorithm(init_pop, n_gen)
print('Melhor configuração = {}, melhor valor = {}'.format(best_config, best_value))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [10:32<00:00, 31.61s/it]

Melhor configuração = [203 220 322  75 169 147], melhor valor = 22.9960647594
